## 1st step: scaling

In [ ]:
import subprocess
import os

data_path = "../test_data/test1/optim_data.pkl"
label = "test_scaling"
sub_id = data_path.split('/')[-2]
os.makedirs(f"../optim_result/{label}",exist_ok=True)
os.makedirs(f"../optim_result/{label}/log",exist_ok=True)
os.makedirs(f"../optim_result/{label}/out",exist_ok=True)


# NOTE:
# - `python -u` : unbuffered stdout/stderr (so logs stream in Jupyter)
# - `-m` not needed; just call script directly
command = (
    "python -u ../optim/hippo_lv_trs_train.py "
    f"--data_path {data_path} "
    f"--tag {label} "
    "--learning_rate 0.0005 "
    f"--sub_id {sub_id} "
    "--gpu 3 "
    "--lda 2 0 0 0 0 0 0"
)

print("Running command:\n", command)

# Stream logs live into the notebook
process = subprocess.Popen(
    command,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,   # line-buffered
)

for line in process.stdout:
    print(line, end="")  # already includes newline

return_code = process.wait()
print("\nProcess finished with return code:", return_code)

# Optional: raise error if failed
if return_code != 0:
    raise RuntimeError(f"Command failed with return code {return_code}")

Running command:
 python -u ../optim/hippo_lv_trs_train.py --data_path ../test_data/test2/optim_data.pkl --tag test_scaling --learning_rate 0.0005 --sub_id test2 --gpu 3 --lda 2 0 0 0 0 0 0
Namespace(gpu='3', batch_size=1, epoch=2001, learning_rate=0.0005, num_point=1024, data_path='../test_data/test2/optim_data.pkl', tag='test_scaling', sub_id='test2', lda=[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
test2 !!!!!
Directory '../optim_result/test_scaling/log/test2' created successfully.
dict_keys(['cuttail_vert', 'cuttail_lv', 'cuttail_hippo', 'tri1', 'tri2', 'tri3'])
@@@@@!! test2
torch.Size([5117, 3]) !! torch.Size([2730, 3])
lambda is following. 
 1. pm: 2.0
 2. cf 0.0
 3. edge 0.0
 4. lap 0.0
 5. norm_con 0.0
 6. l2_vert 0.0
 7.l2_norm 0.0
Start training...

  0%|          | 0/2001 [00:00<?, ?it/s]0 {'loss': tensor(758.6214, device='cuda:0', grad_fn=<AddBackward0>), 'point_mesh_dist_loss': tensor(379.3107, device='cuda:0'), 'chamfer_loss': tensor(240.6585, device='cuda:0'), 'edge_loss': tens

## 1st step: Optimization Result

In [4]:
import k3d
import pickle
import numpy as np
import glob
from tqdm import tqdm
%matplotlib inline

file = '../test_data/test1/optim_data.pkl'

with open("../temp_meshes/cuttail_temp_L_tex.pkl", "rb") as f:
    data = pickle.load(f)
with open(file, "rb") as f:
    pt_data = pickle.load(f)
id = file.split("/")[-2]
scaler = np.load(glob.glob(f"../optim_result/test_scaling/out/{id}/1000_*.npy")[-1])
print(scaler)

lv_pt = pt_data["lv"]
hippo_pt = pt_data["hippo"]
vert= data['cuttail_vert']
tex1_pt = pt_data["tex1"]
tex2_pt = pt_data["tex2"]
tex3_pt = pt_data["tex3"]

# move to origin2
for d in [hippo_pt, tex1_pt, tex2_pt, tex3_pt]:
    d[:,0] -=lv_pt[:,0].min()
vert[:,0] = vert[:,0] - vert[:,0].min()
lv_pt[:,0] -= lv_pt[:,0].min()

for d in [hippo_pt, tex1_pt, tex2_pt, tex3_pt]:
    d[:,1] -=lv_pt[:,1].max()
vert[:,1] = vert[:,1] - vert[:,1].max()
lv_pt[:,1] -= lv_pt[:,1].max()

tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
for d in [hippo_pt, tex1_pt, tex2_pt, tex3_pt]:
    d[:,2] -= (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2
lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2


# scaling of template mesh
vert *= scaler[0,0,:]

# Save target shape
tgt_pts = {"lv": lv_pt, "hippo": hippo_pt, "vert": vert, "tex1":tex1_pt, "tex2":tex2_pt, "tex3":tex3_pt}
with open(file.replace("optim_data","scale_optim_data"), "wb") as f:
    pickle.dump(tgt_pts, f)

plot = k3d.plot()
plot.display()
plot += k3d.points(lv_pt, point_size=1, color = 0x3f6bc5)
plot += k3d.points(hippo_pt, point_size=1, color = 0x00ff00, opacity = 0.3)
plot += k3d.points(tex1_pt, point_size=1, color = 0xabffab, opacity = 0.3)
plot += k3d.points(tex2_pt, point_size=1, color = 0xab2950, opacity = 0.3)
plot += k3d.points(tex3_pt, point_size=1, color = 0xf43344, opacity = 0.3)
plot += k3d.mesh(vert, data['tri1'], opacity = 0.3, color = 0xabffab)
plot += k3d.mesh(vert, data['tri2'], opacity = 0.3, color = 0xab2950)
plot += k3d.mesh(vert, data['tri3'], opacity = 0.3, color = 0xf43344)
plot += k3d.mesh(vert, data['cuttail_lv'], opacity = 0.7, color = 0xf43344)

[[[1.1420989 1.196615  1.0770763]]]


Output()

## 2nd step: deformable optimization

In [ ]:
import subprocess
import os

data_path = "../test_data/test1/scale_optim_data.pkl"
label = "test_final_optim"
sub_id = data_path.split('/')[-2]
os.makedirs(f"../optim_result/{label}",exist_ok=True)
os.makedirs(f"../optim_result/{label}/log",exist_ok=True)
os.makedirs(f"../optim_result/{label}/out",exist_ok=True)


# NOTE:
# - `python -u` : unbuffered stdout/stderr (so logs stream in Jupyter)
# - `-m` not needed; just call script directly
command = (
    "python -u ../optim/hippo_lv_tex_train2.py "
    f"--data_path {data_path} "
    f"--tag {label} "
    "--learning_rate 0.0005 "
    f"--sub_id {sub_id} "
    "--gpu 3 "
    "--lda 2 2 2000 100 500 1 1"
)

print("Running command:\n", command)

# Stream logs live into the notebook
process = subprocess.Popen(
    command,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,   # line-buffered
)

for line in process.stdout:
    print(line, end="")  # already includes newline

return_code = process.wait()
print("\nProcess finished with return code:", return_code)

# Optional: raise error if failed
if return_code != 0:
    raise RuntimeError(f"Command failed with return code {return_code}")

Running command:
 python -u ../optim/hippo_lv_tex_train2.py --data_path ../test_data/test1/scale_optim_data.pkl --tag test_final_optim --learning_rate 0.0005 --sub_id test1 --gpu 3 --lda 2 2 2000 100 500 1 1
Namespace(gpu='3', batch_size=1, epoch=5001, learning_rate=0.0005, num_point=1024, data_path='../test_data/test1/scale_optim_data.pkl', tag='test_final_optim', sub_id='test1', lda=[2.0, 2.0, 2000.0, 100.0, 500.0, 1.0, 1.0])
test1 !!!!!
Directory '../optim_result/test_final_optim/log/test1' created successfully.
Start training...
@@@@@!! test1
tensor(5.1481, device='cuda:0') tensor(55.7969, device='cuda:0') BASIC LOSS
lambda is following. 
 1. pm: 2.0
 2. cf 2.0
 3. edge 2000.0
 4. lap 100.0
 5. norm_con 500.0
 6. l2_vert 1.0
 7.l2_norm 1.0

  0%|          | 0/5001 [00:00<?, ?it/s]0 {'loss': tensor(252.4445, device='cuda:0', grad_fn=<AddBackward0>), 'point_mesh_dist_loss': tensor(16.5914, device='cuda:0'), 'chamfer_loss': tensor(18.9772, device='cuda:0'), 'tri_loss': tensor(62.5593,

In [ ]:
import k3d
import pickle
import numpy as np
import glob
from tqdm import tqdm
%matplotlib inline

file = f'../test_data/{id}/scale_optim_data.pkl'
vert = np.load(glob.glob(f'../optim_result/test_final_optim/out/{id}/smallest*.npy')[0])[0]

with open("../temp_meshes/cuttail_temp_L_tex.pkl", "rb") as f:
    data = pickle.load(f)
with open(file, "rb") as f:
    pt_data = pickle.load(f)
print(pt_data.keys()) #dict_keys(['lv', 'hippo', 'vert', 'tex1', 'tex2', 'tex3'])


plot = k3d.plot()
plot.display()
plot += k3d.points(pt_data['lv'], point_size=1, color = 0x3f6bc5)
plot += k3d.points(pt_data['hippo'], point_size=1, color = 0x00ff00, opacity = 0.3)
plot += k3d.points(pt_data['tex1'], point_size=1, color = 0xabffab, opacity = 0.3)
plot += k3d.points(pt_data['tex2'], point_size=1, color = 0xab2950, opacity = 0.3)
plot += k3d.points(pt_data['tex3'], point_size=1, color = 0xf43344, opacity = 0.3)
plot += k3d.mesh(vert, data['tri1'], opacity = 0.3, color = 0xabffab)
plot += k3d.mesh(vert, data['tri2'], opacity = 0.3, color = 0xab2950)
plot += k3d.mesh(vert, data['tri3'], opacity = 0.3, color = 0xf43344)
plot += k3d.mesh(vert, data['cuttail_lv'], opacity = 0.7, color = 0xf43344)

dict_keys(['lv', 'hippo', 'vert', 'tex1', 'tex2', 'tex3'])


Output()

/opt/conda/envs/pytorch3d/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(
